In [3]:

# Download Data from Yahoo Finance
import datetime as dt
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
# Pour lire les données de Yahoo Finance, vous devez installer le toolbox yfinance
import yfinance as yf 

def get_dow():
    start_sp = dt.datetime(2004,12,31)
    end_sp = dt.datetime(2019, 12,31)

    # Dow Jones index
    # 1st Dow Jones Index + 30 components
    dow30 = ['^DJI','AXP', 'AAPL', 'BA', 'CAT', 'CSCO', 'CVX', 'DD', 'XOM',
            'GE', 'GS', 'HD', 'IBM', 'INTC', 'JNJ', 'KO', 'JPM',
            'MCD', 'MMM', 'MRK', 'MSFT', 'NKE', 'PFE', 'PG',
            'TRV', 'UNH', 'UTX', 'VZ', 'V', 'WMT', 'DIS']

    dow_data = yf.download(dow30, start=start_sp, end=end_sp,group_by="ticker")

    return dow_data

In [158]:
def calculate_fivedays_roi(stock_price, date):
    roi = None
    try:
        past_date = date - np.timedelta64(5, "D")
        current_value = stock_price.loc[date]
        past_value = stock_price.loc[past_date]
        roi = (current_value - past_value)/past_value * 100        
    except KeyError:
        pass        
    return roi

def download_stock_prices(tickers, end_date, period):
    start_date = end_date - np.timedelta64(period, "D")
    #download le prix du stock des compagnies sur l'intervalle de 255 jours (250 jours de rendemment nécessitant 5 jours de recul)
    stock_prices = yf.download(tickers, start=start_date, end=end_date,group_by="ticker")
    return stock_prices.transpose() 

def get_roi(stock_prices, ticker, end_date, period):
    rois = []
    dates = []
    company_stock_price = stock_prices.loc[(ticker, "Close")]

    # calculate de ROI over the last 250 days
    curr_date = end_date - np.timedelta64(period, "D")
    while(curr_date < end_date):
        roi = calculate_fivedays_roi(company_stock_price, curr_date)
        if roi:
            rois.append(roi)
            dates.append(curr_date)
        curr_date = curr_date + np.timedelta64(1, "D")                

    return pd.Series(rois,index=dates)

# var historique sur periode de 250 jours et comparaison 5 jours avants
def historic_var(tickers, date):
    period = 250
    stock_prices = download_stock_prices(tickers, date, period+5)
            
    result = pd.DataFrame()
    for ticker in tickers:             
        result[ticker] = get_roi(stock_prices, ticker, date, period)
    
    return result
    
    

In [159]:
stocks = ['DJI','AXP', 'AAPL', 'BA', 'CAT', 'CSCO', 'CVX', 'DD', 'XOM',
            'GE', 'GS', 'HD', 'IBM', 'INTC', 'JNJ', 'KO', 'JPM',
            'MCD', 'MMM', 'MRK', 'MSFT', 'NKE', 'PFE', 'PG',
            'TRV', 'UNH', 'UTX', 'VZ', 'V', 'WMT', 'DIS']
a = historic_var(stocks,np.datetime64("2019-03-01"))


[*********************100%***********************]  31 of 31 completed


In [160]:
a

,DJI,AXP,AAPL,BA,CAT,CSCO,CVX,DD,XOM,GE,...,NKE,PFE,PG,TRV,UNH,UTX,VZ,V,WMT,DIS
2018-06-25,-1.642482,2.198717,-2.321717,-3.352882,-4.576261,-3.315045,-2.139038,0.136595,-0.882535,-1.009317,...,-3.171845,-0.219413,2.557683,-1.016079,-0.830660,-0.598277,3.139942,-3.372699,3.420644,-2.519836
2018-06-26,-0.730119,1.635105,-0.555383,-1.883556,-2.859604,-1.413347,1.280698,1.577816,1.192116,7.680249,...,-1.866385,0.220391,2.040813,-0.672332,-0.841635,1.749268,1.377747,-1.471788,2.101893,-1.539331
2018-06-27,-1.884800,-0.277696,-0.410986,-2.682127,-3.808776,-2.013886,0.719426,-2.074622,0.405507,6.973180,...,-2.832632,-0.821691,0.335790,-2.467653,-2.437393,-0.408426,-0.904335,-3.184806,2.440462,-2.238102
2018-07-02,0.786146,2.103961,1.639872,1.897999,1.248044,1.133946,-1.388889,0.243864,0.061196,-4.226359,...,9.810792,0.331408,0.566347,0.865340,-0.278036,0.538757,2.251058,1.129595,-3.326044,1.317817
2018-07-03,-0.170132,0.897180,-0.851753,-0.513970,-1.617288,-0.628785,-0.740093,-1.511717,0.329385,-3.326101,...,6.387729,-0.027509,-0.192186,0.261331,0.849327,0.256822,-0.314589,-0.971831,-1.653853,-0.696760
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019-02-19,NaN,-0.112552,0.076106,1.571422,2.397834,2.582645,0.973251,3.665020,2.569827,0.896423,...,-0.129902,1.501066,1.553929,1.709608,1.914603,3.633289,2.609661,0.894104,3.735283,2.575455
2019-02-20,0.275086,-0.558762,0.944725,0.856518,3.017622,0.404614,0.779221,3.687319,1.055205,0.099103,...,-0.632468,-0.471700,0.812343,0.776759,-0.597498,0.524413,1.087742,-0.124221,-0.110012,0.968118
2019-02-25,0.530159,0.140477,1.278845,1.262011,0.783983,2.337296,-0.548716,-0.284494,-0.038200,7.128711,...,0.388970,2.085311,0.292104,2.119624,-1.821826,-0.124572,1.829269,0.918954,-0.760908,-0.079173
2019-02-26,0.805815,0.309541,1.911612,2.200777,-0.648048,3.582272,0.671481,1.121964,1.079419,10.532400,...,2.203700,2.088280,0.050113,2.657523,-1.283777,0.688729,0.872659,2.111106,-0.704293,-0.691225
